In [1]:
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

import sys
from pathlib import Path

PROJECT_ROOT = Path("/Users/yuqiugan/Desktop/Personal_DS_Projects/house_rental_agent")
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.GoogleMaps_tools import query_google_maps
from src.settings import GOOGLE_GEMINI_API_KEY


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_GEMINI_API_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

tools = [query_google_maps]

system_prompt = """
You are a transit planning assistant.
Return structured blocks when helpful.
You have only one tool to use 'query_google_maps' which can check real-time information.
Always include a brief explanation block built from data (no private thoughts). 
If you have used the tool, tell me what tool you used and if there are some option that the tool has, tell me what option you used.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

rental_agent = create_tool_calling_agent(llm, tools, prompt)
rental_agent_executor = AgentExecutor(
    agent = rental_agent, tools = tools, verbose=False, return_intermediate_steps=True
)

In [4]:
query_message = "Help me to check the route from '155 Goldsborough Dr, Bayonne, NJ 07002' to '277 Park Ave, New York, NY 10172'"
response = rental_agent_executor.invoke({"input": query_message})

In [7]:
type(response)

dict

In [8]:
response.keys()

dict_keys(['input', 'output', 'intermediate_steps'])

In [9]:
response.get('input')

"Help me to check the route from '155 Goldsborough Dr, Bayonne, NJ 07002' to '277 Park Ave, New York, NY 10172'"

In [11]:
response.get('output')

'The total distance is 11.6 miles and it will take 1 hour and 28 minutes.\n\nHere are the steps:\n1. Walk to 34th Street.\n2. Take the Light rail towards Terminal from 34th Street to Exchange Place.\n3. Walk to Exchange Place.\n4. Take the Subway towards World Trade Center from Exchange Place to World Trade Center.\n5. Walk to Fulton St.\n6. Take the Subway towards Eastchester-Dyre Av from Fulton St to Grand Central-42 St.\n7. Walk to 277 Park Ave, New York, NY 10172, USA.\n\nI used the `query_google_maps` tool with the `generate_route_map` option set to `True`.'

In [10]:
response.get('intermediate_steps')

[(ToolAgentAction(tool='query_google_maps', tool_input={'generate_route_map': True, 'origin': '155 Goldsborough Dr, Bayonne, NJ 07002', 'destination': '277 Park Ave, New York, NY 10172', 'origin_name': '155 Goldsborough Dr, Bayonne, NJ 07002'}, log="\nInvoking: `query_google_maps` with `{'generate_route_map': True, 'origin': '155 Goldsborough Dr, Bayonne, NJ 07002', 'destination': '277 Park Ave, New York, NY 10172', 'origin_name': '155 Goldsborough Dr, Bayonne, NJ 07002'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'query_google_maps', 'arguments': '{"generate_route_map": true, "origin": "155 Goldsborough Dr, Bayonne, NJ 07002", "destination": "277 Park Ave, New York, NY 10172", "origin_name": "155 Goldsborough Dr, Bayonne, NJ 07002"}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a038399b-1b30-4430-9720-13afdfb98153', tool_calls=[{'name': 'query_google_maps', 'args': {'

In [6]:
from src.GoogleMaps_tools import best_transit_route, static_map_with_route
import re

route = best_transit_route("155 Goldsborough Dr, Bayonne, NJ 07002",
                           "277 Park Ave, New York, NY 10172")
fn = re.sub(r'[^A-Za-z0-9._-]+', '_', "155 Goldsborough Dr, Bayonne, NJ 07002").strip('_')
res = static_map_with_route(route, origin_name=fn)
print(res)

{'status': 'ok', 'path': '/Users/yuqiugan/Desktop/Personal_DS_Projects/house_rental_agent/data/Google_route/155_Goldsborough_Dr_Bayonne_NJ_07002.png'}
